In [1]:
using ProgressMeter

In [7]:
path = "/Users/zahedi/Dropbox/Raphael\ -\ Keyan/797798d9-d347-447a-b1c7-56c15fe3baba/"

dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)

w_indices = "18,19,20,33,34,35,48,49,50,63,64,65,93,94,95"
a_indices = "8,9,10,11"
wbins     = 500
abins     = 100

100

### 1) Preprocessing the files

In [3]:
function copy_and_convert_hand(path)

    in_dirs  = map(x->"$path/$x/raw", dirs)
    out_dirs = map(x->"$path/$x/analysis", dirs)

    p = Progress(length(in_dirs)*1, 1, "hand.sofastates.csv ...")

    r = collect(1:7:224)
    s = collect(2:7:224)
    t = collect(3:7:224)
    r = vcat(r',s',t')
    r = vec(r)

    for index in 1:length(in_dirs)
        if isfile("$(in_dirs[index])/hand.sofastates.txt")
            a = readcsv("$(in_dirs[index])/hand.sofastates.txt")[2:2:end,:]
            a = map(x->replace(x,"  X= ",""),a)
            m = nothing
            for j = 1:length(a)
                b = a[j]
                c = split(b, " ")
                d = map(x->parse(Float64,x), [d for d in c])
                f = convert(Array{Float64,1},d)'
                if m == nothing
                    m = f[r]'
                else
                    m = vcat(m, f[r]')
                end
            end

            for j = 1:size(m)[1]
                for k = 5:3:size(m)[2]
                    m[j,k] -= m[j,2]
                end
                m[j,2] = 0.0
            end
            writecsv("$(out_dirs[index])/hand.sofastates.csv",m)
        end
        next!(p)
    end
end
   
function copy_and_convert_control(path)
    dirs = readdir(path)
    dirs = filter(x->contains(x,"hand"), dirs)

    in_dirs  = map(x->"$path/$x/raw", dirs)
    out_dirs = map(x->"$path/$x/analysis", dirs)

    p = Progress(length(in_dirs), 1, "control.states.csv ...")

    for i in 1:length(in_dirs)
        if isfile("$(in_dirs[i])/control.states.csv")
            a = readcsv("$(in_dirs[i])/control.states.csv")[2:end-1,:]
            a = map(x->Float64(x),a)
            writecsv("$(out_dirs[i])/control.states.csv",a)
        end
        next!(p)
    end
end

function copy_and_convert_files(path)
    copy_and_convert_hand(path)
    copy_and_convert_control(path)
end

copy_and_convert_files (generic function with 1 method)

In [4]:
copy_and_convert_files(path)

hand.sofastates.csv ...100% Time: 0:00:35
control.states.csv ...100% Time: 0:00:05


### 2)  Get Min/Max Values

In [8]:
function write_domains(path)    
    in_dirs      = map(x->"$path/$x/analysis", dirs)
    controlfiles = map(x->"$x/control.states.csv",in_dirs)
    a_domains    = map(x->"$x/A.domains.csv",in_dirs)
    handfiles    = map(x->"$x/hand.sofastates.csv",in_dirs)
    w_domains    = map(x->"$x/W.domains.csv",in_dirs)

    function minmax(in_,out_)
        a = readcsv(in_)
        b = minimum(a,1)
        c = maximum(a,1)
        d = vcat(b,c)
        writecsv(out_, d)
    end

    p = Progress(length(in_dirs), 1, "domain files ...")

    for i = 1:length(in_dirs)
        if isfile(controlfiles[i])
            minmax(controlfiles[i], a_domains[i])
        end
        if isfile(handfiles[i])
            minmax(handfiles[i],w_domains[i])
        end
        next!(p)
    end
end

write_domains(path)

domain files ...100% Time: 0:00:01


### 3)  Hand, Controller, Objects, Positions

In [9]:
in_dirs = map(x->"$path/$x/analysis", dirs)

fd = open("Makefile.1","w")
write(fd,"all:\n")
for d in in_dirs
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$d\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.1`)

/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//hand0_0-controller0-objectbox_-10.0_-10.0_-15.0/analysis"
MC: 4.0892
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//hand0_0-controller0-objectbox_-10.0_-10.0_0.0/analysis"
MC: 4.22999
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//hand0_0-controller0-objectbox_-10.0_-10.0_15.0/analysis"
MC: 3.85
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 1

### 4) Hand, Controller, Objects, Average over positions

In [10]:
parents = map(x->replace(x, r"^(.+object\w+)_.+$", s"\g<1>"),dirs)
parents = unique(parents)

for d in parents
    mkpath("$path/combined/object_position/$d/analysis")
end

pg = Progress(length(parents), 1, "combine hand.sofastates.csv and control.states.csv ...")

for p in parents
    ds = filter(x->contains(x,"$(p)_"),dirs)
    hand = nothing
    control = nothing
    
    for a in ds
        pp = "$path/$a/analysis/hand.sofastates.csv"
        if isfile(pp)
            h = readcsv(pp)
            if hand == nothing
                hand = h
            else
                hand = vcat(hand,h)
            end
        else
            println(">  missing $pp")
        end

        pp = "$path/$a/analysis/control.states.csv"
        if isfile(pp)
            c = readcsv(pp)
            if control == nothing
                control = c
            else
                control = vcat(control,c)
            end
        else
            println(">  missing $pp")
        end
    end
    if hand != nothing
        writecsv("$path/combined/object_position/$p/analysis/hand.sofastates.csv",hand)
    end
    if control != nothing
        writecsv("$path/combined/object_position/$p/analysis/control.states.csv", control)
    end
    next!(pg)
end

combine hand.sofastates.csv and control.states.csv ...100% Time: 0:00:04


In [23]:
write_domains("$path/combined/object_position/")

domain files ...100% Time: 0:00:01


In [11]:
parents = map(x->replace(x, r"^(.+object\w+)_.+$", s"\g<1>"),dirs)
parents = unique(parents)

fd = open("Makefile.2","w")
write(fd,"all:\n")
for p in parents
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$path/combined/object_position/$p/analysis\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.2`)

/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/object_position/hand0_0-controller0-objectbox/analysis"
MC: 8.08738
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/object_position/hand0_0-controller0-objectcylinder/analysis"
MC: 7.93667
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/object_position/hand0_0-controller0-objectsphere/analysis"
MC: 7.91961
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,

### 5) Hand, Controller, Avg over Objects, Position = 0.0, 0.0, 0.0

In [25]:
dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)
dirs = filter(x->contains(x,"_0.0_0.0_0.0"), dirs)

parents = map(x->replace(x, r"^(.+)-object\w+_0.0_0.0_0.0", s"\g<1>"),dirs)
parents = unique(parents)

for d in parents
    mkpath("$path/combined/object_position_0/$d/analysis")
end

for p in parents
    println(p)
    ds = filter(x->contains(x,"$(p)"),dirs)
    hand = nothing
    control = nothing
    
    for a in ds
        pp = "$path/$a/analysis/hand.sofastates.csv"
        #println(pp)
        #println(a)
        if isfile(pp)
            h = readcsv(pp)
            if hand == nothing
                hand = h
            else
                hand = vcat(hand,h)
            end
        else
            println(">  missing $pp")
        end

        pp = "$path/$a/analysis/control.states.csv"
        if isfile(pp)
            c = readcsv(pp)
            if control == nothing
                control = c
            else
                control = vcat(control,c)
            end
        else
            println(">  missing $pp")
        end
    end
    if hand != nothing
        writecsv("$path/combined/object_position_0/$p/analysis/hand.sofastates.csv",hand)
    end
    if control != nothing
        writecsv("$path/combined/object_position_0/$p/analysis/control.states.csv", control)
    end
    next!(pg)
end

write_domains("$path/combined/object_position_0/")

dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)
dirs = filter(x->contains(x,"_0.0_0.0_0.0"), dirs)

parents = map(x->replace(x, r"^(.+)-object\w+_0.0_0.0_0.0", s"\g<1>"),dirs)
parents = unique(parents)

fd = open("Makefile.3","w")
write(fd,"all:\n")
for p in parents
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$path/combined/object_position_0/$p/analysis\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.3`)

hand0_0-controller0
hand1_0-controller0
hand2_0-controller0
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/object_position_0/hand0_0-controller0/analysis"
MC: 5.38989
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/object_position_0/hand1_0-controller0/analysis"
MC: 5.02456
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/object_position_0/hand2_0-controller0/analysis"
MC: 5.26768


### 6) Hand, Controller, Avg over Objects, Avg Position

In [26]:
dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)

parents = map(x->replace(x, r"^(.+)-object.+", s"\g<1>"),dirs)
parents = unique(parents)

for d in parents
    mkpath("$path/combined/hand_control/$d/analysis")
end

for p in parents
    println(p)
    ds = filter(x->contains(x,"$(p)"),dirs)
    hand = nothing
    control = nothing
    
    for a in ds
        pp = "$path/$a/analysis/hand.sofastates.csv"
        #println(pp)
        #println(a)
        if isfile(pp)
            h = readcsv(pp)
            if hand == nothing
                hand = h
            else
                hand = vcat(hand,h)
            end
        else
            println(">  missing $pp")
        end

        pp = "$path/$a/analysis/control.states.csv"
        if isfile(pp)
            c = readcsv(pp)
            if control == nothing
                control = c
            else
                control = vcat(control,c)
            end
        else
            println(">  missing $pp")
        end
    end
    if hand != nothing
        writecsv("$path/combined/hand_control/$p/analysis/hand.sofastates.csv",hand)
    end
    if control != nothing
        writecsv("$path/combined/hand_control/$p/analysis/control.states.csv", control)
    end
    next!(pg)
end

write_domains("$path/combined/hand_control/")

dirs = readdir(path)
dirs = filter(x->contains(x,"hand"), dirs)

parents = map(x->replace(x, r"^(.+)-object.+", s"\g<1>"),dirs)
parents = unique(parents)

fd = open("Makefile.4","w")
write(fd,"all:\n")
for p in parents
    command = "/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$path/combined/hand_control/$p/analysis\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.4`)

hand0_0-controller0
hand1_0-controller0
hand2_0-controller0
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/hand_control/hand0_0-controller0/analysis"
MC: 9.16195
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/hand_control/hand1_0-controller0/analysis"
MC: 8.78835
/Users/zahedi/projects/builds/entropy-build/bin/rbo_mc -wi 18,19,20,33,34,35,48,49,50,63,64,65,93,94,95 -ai 8,9,10,11 -wbins 500 -abins 100 -d "/Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//combined/hand_control/hand2_0-controller0/analysis"
MC: 9.36642


### Auswertung

In [27]:
dirs = readdir("$path/combined/hand_control")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/combined/hand_control/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/combined/hand_control/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

In [28]:
dirs = readdir("$path/combined/object_position")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/combined/object_position/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/combined/object_position/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

In [29]:
dirs = readdir("$path/combined/object_position_0")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/combined/object_position_0/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/combined/object_position_0/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

In [30]:
dirs = readdir("$path")
in_dirs = filter(x->contains(x,"hand"), dirs)

mc_files = map(x->"$path/$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", in_dirs)
values   = map(x->readcsv(x)[1],mc_files)
fd = open("$path/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    write(fd, "$(in_dirs[i]),$(values[i])\n")
end
close(fd)

LoadError: LoadError: SystemError: opening file /Users/zahedi/Dropbox/Raphael - Keyan/797798d9-d347-447a-b1c7-56c15fe3baba//hand0_0-controller0-objectbox_-10.0_-10.0_-15.0/analysis/mc_w-averaged_500_100.txt: No such file or directory
while loading In[30], in expression starting on line 5